# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [2]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 2s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [3]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 14, 9, 66, 66, 78, 2, 287, 43, 89, 6, 333, 967, 284, 127, 27, 292, 21, 51, 166, 12, 290, 149, 9, 4, 136, 121, 2, 2, 2, 6, 1486, 1002, 34, 4, 3681, 17, 48, 8, 516, 98, 137, 4, 2, 299, 6, 613, 19, 4, 2, 13, 188, 4, 2314, 23, 25, 166, 72, 181, 8, 842, 4, 360, 7, 4, 4176, 2, 8, 628, 2, 5, 2814, 2, 21, 13, 426, 169, 12, 23, 2, 740, 44, 319, 83, 4, 705, 99, 163, 18, 715, 4, 360, 7, 4, 20, 9, 2441, 4, 603, 5, 690, 26, 53, 221, 74, 4, 229, 2, 4, 755, 63, 2, 4, 888, 7, 138, 36, 62, 81, 6, 755, 33, 32, 61, 1897, 16, 15, 2189, 2, 3044, 6, 176, 7, 134, 503, 102, 152, 135, 76, 18, 4, 3446, 829, 2]
--- Label ---
0


Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [4]:
a = {}
a['x'] =2
a['y'] =3
print(a.items())
print(a.get('x'))

dict_items([('x', 2), ('y', 3)])
2


In [5]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 1s 0us/step
--- Review (with words) ---
['the', 'as', 'it', 'had', 'had', 'do', 'and', 'worth', 'out', "don't", 'is', 'father', 'business', 'shows', 'end', 'be', 'together', 'not', 'when', 'find', 'that', 'main', "doesn't", 'it', 'of', 'scenes', 'know', 'and', 'and', 'and', 'is', 'putting', 'outside', 'who', 'of', 'creators', 'movie', 'what', 'in', 'car', 'any', 'go', 'of', 'and', 'effects', 'is', 'annoying', 'film', 'of', 'and', 'was', "can't", 'of', 'disgusting', 'are', 'have', 'find', 'we', 'pretty', 'in', 'whatever', 'of', 'hollywood', 'br', 'of', 'exotic', 'and', 'in', 'english', 'and', 'to', 'flight', 'and', 'not', 'was', 'against', 'same', 'that', 'are', 'and', 'giving', 'has', 'wife', 'first', 'of', 'thriller', 'movies', 'makes', 'but', 'jack', 'of', 'hollywood', 'br', 'of', 'on', 'it', 'display', 'of', 'simple', 'to', '7', 'he', 'up', 'done', 'been', 'of', 'guy', 'and', 'of', "'", 'really', 'and', 'of', '20', 'br', 'such', 'fr

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

...


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [6]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [7]:
X_train[200]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embeding_size =32
model = Sequential()
model.add(Embedding(vocabulary_size, embeding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

...

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [9]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [16]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 5


# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2,y_train2 = X_train[batch_size:], y_train[batch_size:]
# TODO: Train your model
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size = batch_size, epochs=num_epochs)

Train on 24936 samples, validate on 64 samples
Epoch 1/5
24936/24936 [==============================] - 535s 21ms/step - loss: 0.2227 - acc: 0.9151 - val_loss: 0.4016 - val_acc: 0.8125
Epoch 2/5
24936/24936 [==============================] - 543s 22ms/step - loss: 0.2329 - acc: 0.9076 - val_loss: 0.3779 - val_acc: 0.8281
Epoch 3/5
24936/24936 [==============================] - 549s 22ms/step - loss: 0.1956 - acc: 0.9243 - val_loss: 0.3754 - val_acc: 0.8281
Epoch 4/5
24936/24936 [==============================] - 558s 22ms/step - loss: 0.1651 - acc: 0.9386 - val_loss: 0.4422 - val_acc: 0.8125
Epoch 5/5
24936/24936 [==============================] - 552s 22ms/step - loss: 0.1582 - acc: 0.9427 - val_loss: 0.6144 - val_acc: 0.7969


In [17]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
import os
cache_dir = os.path.join("cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [18]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.8646


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!